In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite


In [2]:
pd.read_csv('/kaggle/input/amazon-fine-food-reviews/Reviews.csv')

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [3]:
# 讀取資料
df = pd.read_csv('/kaggle/input/amazon-fine-food-reviews/Reviews.csv', usecols=['Text', 'Score'])
df = df.head(10000)

# 顯示前 50 行
df.head(50)

# 或顯示最後 20 行
#df.tail(20)

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...
5,4,I got a wild hair for taffy and ordered this f...
6,5,This saltwater taffy had great flavors and was...
7,5,This taffy is so good. It is very soft and ch...
8,5,Right now I'm mostly just sprouting this so my...
9,5,This is a very healthy dog food. Good for thei...


In [4]:
# 進行二元分類
df['Sentiment'] = df['Score'].apply(lambda x: 1 if x >= 4 else 0)
df = df[['Text', 'Sentiment']]

df.head(50)

,Text,Sentiment
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,1
5,I got a wild hair for taffy and ordered this f...,1
6,This saltwater taffy had great flavors and was...,1
7,This taffy is so good. It is very soft and ch...,1
8,Right now I'm mostly just sprouting this so my...,1
9,This is a very healthy dog food. Good for thei...,1


In [5]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# 可以選擇擴展停頓詞
stop_words = list(ENGLISH_STOP_WORDS) + ['additional', 'words']

# 使用 pandas 的方法將 list 轉換為 Series
stop_words_series = pd.Series(stop_words)

# 顯示停頓詞
stop_words_series.head(50)

0             each
1           behind
2              out
3     nevertheless
4              per
5             must
6              are
7           detail
8          hundred
9           cannot
10             cry
11          amount
12              it
13            side
14         because
15            they
16              re
17          should
18          seemed
19           sixty
20           alone
21            ever
22             too
23          except
24           other
25             six
26            well
27             own
28            upon
29             all
30          latter
31       meanwhile
32              as
33         between
34             top
35         nothing
36            fill
37            with
38          across
39          eleven
40             who
41          myself
42             and
43          become
44           front
45        moreover
46            mill
47         whether
48           about
49           empty
dtype: object

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 向量化
tfidf = TfidfVectorizer(stop_words=stop_words)
X_tfidf = tfidf.fit_transform(df['Text'])


In [7]:
from gensim.models import Word2Vec
# 假設已經對Text進行了tokenization
sentences = [text.split() for text in df['Text']]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df['Sentiment'], test_size=0.2, random_state=42)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)


RandomForestClassifier()

In [9]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_tfidf, df['Sentiment'], cv=4, scoring='accuracy')
print("Cross-validation accuracy: ", scores.mean())


Cross-validation accuracy:  0.8043
